In [ ]:
import os
import time
import yaml
import pickle
import numpy as np
import subprocess
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import sys
from pathlib import Path
import json
import datetime
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker
import multiprocessing
from scipy.optimize import curve_fit
project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
os.chdir(str(project_dir))
from tools.benchmarks.helpers import *
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot
from matplotlib import rc

In [ ]:
executable_path = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench"
def get_success_prob (algorithm, ntx, nty, num_target, loading_efficiency = 0.6, alpha = 0.985, nu = 0.985, lifetime = 60, num_trials = 100, num_reps = 10, batched = 1):
    command =  f"{executable_path} {algorithm} {ntx} {nty} {num_target} {loading_efficiency} {alpha} {nu} {lifetime} {num_trials} {num_reps} {batched}" 
    return os.popen(command).read()

### Get Data

In [ ]:
# Lifetime vs Batched/Unbatched 
life_times = range(1,121) 
algorithms = ["REDREC-CPU-V3-2D"]
batched = [True, False]
problem_size = [(32, y) for y in range(58, 73, 2)]
target = 32 * 32

success_prob = []
for algorithm in algorithms:
    for ntx, nty in problem_size:
        for batch in batched:
            for lifetime in life_times:
                result = get_success_prob(algorithm, ntx, nty, target, lifetime = lifetime, batched = int(batch))
                success_prob.append(((algorithm, ntx, nty, target, lifetime, batched), float(result)))
                print(f"Algorithm: {algorithm}, Problem Size: {ntx}x{nty}, Batched: {batch}, Lifetime: {lifetime}, Success Probability: {result}")
filename = str(project_dir) + "/resources/operational_benchmark/lifetime_batched_unbatched" + time.time() + ".pkl"
with open(filename, "wb") as f:
    pickle.dump(success_prob, f)

In [ ]:
# RedRec vs BIRD vs ARO
algorithms = ["REDREC-CPU-V3-2D", "BIRD-CPU-2D", "ARO-2D"]
problem_sizes = [(32, y) for y in range (50, 80, 2)]
target = 32*32
success_prob = []
for algorithm in algorithms:
    for ntx, nty in problem_sizes:
        result = get_success_prob(algorithm, ntx, nty, target)
        success_prob.append(((algorithm, ntx, nty, target), float(result)))
filename = str(project_dir) + "/resources/operational_benchmark/redrec_bird_aro" + time.time() + ".pkl"
with open(filename, "wb") as f:
    pickle.dump(success_prob, f)

In [ ]:
# RedRec Heatmap
algorithms = ["REDREC-CPU-V3-2D"]
problem_sizes = [(x, y) for x in range(2, 65, 2) for y in range(2, 65, 2)]
target = 32*32
success_prob = []
for algorithm in algorithms:
    for ntx, nty in problem_sizes:
        result = get_success_prob(algorithm, ntx, nty, target)
        success_prob.append(((algorithm, ntx, nty, target), float(result)))
filename = str(project_dir) + "/resources/operational_benchmark/redrec_heatmap" + time.time() + ".pkl"
with open(filename, "wb") as f:
    pickle.dump(success_prob, f)

In [ ]:
# Bird Heatmap
algorithms = ["BIRD-CPU-2D"]
problem_sizes = [(x, y) for x in range(2, 65, 2) for y in range(2, 65, 2)]
target = 32*32
success_prob = []
for algorithm in algorithms:
    for ntx, nty in problem_sizes:
        result = get_success_prob(algorithm, ntx, nty, target)
        success_prob.append(((algorithm, ntx, nty, target), float(result)))
filename = str(project_dir) + "/resources/operational_benchmark/bird_heatmap" + time.time() + ".pkl"
with open(filename, "wb") as f:
    pickle.dump(success_prob, f)

### Generate Operational Benchmarking S-Shape Curve ###

In [ ]:
# If plotting multiple saved datasets which are being read from pickle files, use this cell

with open('/home/tqtraaqs1/LLRS/resources/operational-benchmarking/redrec_results_batched_1723510576.0348651.pkl', 'rb') as f:
    data_1 = pickle.load(f)
    data_1.sort(key = lambda x: x[0])

with open('/home/tqtraaqs1/LLRS/resources/operational-benchmarking/bird_results_batched_1723523223.0269482.pkl', 'rb') as f:
    data_2 = pickle.load(f)
    data_2.sort(key = lambda x: x[0])

with open('/home/tqtraaqs1/LLRS/resources/operational-benchmarking/aro_results_batched_1722574595.6301587.pkl' , 'rb') as f:
    data_3 = pickle.load(f)
    data_3.sort(key = lambda x: x[0])
fig, ax = plt.subplots() 
ax.set_ylabel("Success rate (p)")
ax.set_xlabel("Height of the grid")
prod_plot.production_quality_scatter(fig, ax, [[x[0] for x in data_1], [x[0] for x in data_2], [x[0] for x in data_3]], [[x[1] for x in data_1], [x[1] for x in data_2], [x[1] for x in data_3] ], markers=mark(["rrccpu", "bird", "aro"]), palette=pal(["rrccpu", "bird", "aro"]), save_path=str(project_dir) + "/operational.pdf")
ax.legend(["Red-rec", "Bird", "ARO"],fontsize=4)
print(data_1)
print(data_2)
print(data_3)
yticks(ax,[0, 0.2, 0.4, 0.6, 0.8,1])
xticks(ax,[50, 60,70,80])
plt.savefig(str(project_dir) + "/operational.pdf", format='pdf')
plt.show()



### Bird Surface plot

In [ ]:
list_of_bird_files = [f for f in os.listdir(str(project_dir) + "/resources/operational-benchmarking/bird-loss/")]
list_of_bird_files.sort(key= lambda x: int(x.split("_")[2]))
bird_data = np.zeros(shape=(32, 36))
count = 0
for itr in list_of_bird_files:
    if count == 32:
        break
    with open(str(project_dir) + "/resources/operational-benchmarking/bird-loss/" + itr, 'rb') as f:
        data = pickle.load(f)
        data.sort(key = lambda x: x[0])
        data = [x for x in data if x[0] < 74]
        offset = 36 - len(data)
        zeros = [((x+1)*2,0) for x in range(offset)]
        data = zeros + data
    bird_data[count] = [x[1] for x in data]
    count += 1 

xs = [x for x in range(2, 36, 2)]
ys = [] 
for i in bird_data:
    for (j, z) in zip(i, range(2, 72, 2)):
        if j >= 0.50:
            ys.append(z)
            break

square = np.zeros(shape=(32, 32))
for i in range(32):
    for j in range(32):
        square[i][j] = bird_data[i][j]
W = np.tril(square.T) + np.triu(square, 1)
for i in range(32):
    for j in range(32):
        bird_data[i][j] = W[i][j]

In [ ]:
def func(x, a, m):
    return m *x**a 
model_bird, _ = curve_fit(func, xs, ys)

In [ ]:
styles = ['/home/tqtraaqs1/Experiment/experiment/modules/LLRS/config/TQT-RAAQS.mplstyle']
with plt.style.context(styles):
    fig = plt.figure()
    gs = GridSpec(1,1, figure=fig)
    fig.add_subplot(gs[0, 0])
    ax=fig.axes

    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 'size': 7})
    fontsize = 7
    plt.rc('pdf', fonttype=42) # make the test exportable as text, not an image
    # Calculate figure size in inches and set the size of the figure
    figure_width_cm = 4.3
    figure_height_cm = 4.3
    figure_width_in = figure_width_cm / 2.54
    figure_height_in = figure_height_cm / 2.54
    fig.set_size_inches((figure_width_in, figure_height_in))
    margins = {  #     vvv margin in inches
    "left"   :     0.75 / 4.8,
    "bottom" :     0.5 / 4.8,
    "right"  : 1 - 0.75 / 4.8,
    "top"    : 1 - 0.4   / 4.8 
    }
    fig.subplots_adjust(**margins)
    im = ax[0].imshow(np.rot90(bird_data), extent=[1, 65, 1, 73], cmap='RdBu')
    pixel_size = 0.8 
    fmt.format_plot(ax[0], plot_params={
        'xticks': range(0, 65, 8),  # Centered ticks for x-axis
        'yticks': [x for  x in range(0, 73, 8)],
        'xticklabels': range(0,65,8),
        'yticklabels': [x for x in range(0,73,8)],
        'xlim_epsilon': 0.075,
        'ylim_epsilon': 0.075,
        'xlim': (0,64),
        'ylim': (0,72),
        'ytitle': 'Nty',
        'xtitle': 'Ntx'
    })

    for k in [512, 1024, 2048]:
        xp = np.linspace(k/72, 64, 65)
        yp = k/xp
        fmt.plot_curve(ax[0], xp, yp, series_params={'linestyle': '-', 'primary': 'lightgrey'})
    xcurve = np.linspace(0, 50, 1000, dtype=int)
    y_sp = model_bird[1]*xcurve**model_bird[0]
    fmt.plot_curve(ax[0], xcurve, y_sp, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})
    fmt.plot_curve(ax[0],y_sp, xcurve, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})
 

    divider = fmt.make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cb=fig.colorbar(im, cax=cax, fraction=0.006)
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_label('Mean success probability')

now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
filename = f"bird_mean_success_surface_plot.pdf"
fig.savefig(filename)
plt.show()  


In [ ]:
list_of_bird_files = [f for f in os.listdir(str(project_dir) + "/resources/operational-benchmarking/redrec-v3-loss/")]
list_of_bird_files.sort(key= lambda x: int(x.split("_")[2]))
bird_data = np.zeros(shape=(32, 36))
count = 0
for itr in list_of_bird_files:
    if count == 32:
        break
    with open(str(project_dir) + "/resources/operational-benchmarking/redrec-v3-loss/" + itr, 'rb') as f:
        data = pickle.load(f)
        data.sort(key = lambda x: x[0])
        data = [x for x in data if x[0] < 74]
        offset = 36 - len(data)
        zeros = [((x+1)*2,0) for x in range(offset)]
        data = zeros + data

    bird_data[count] = [x[1] for x in data]
    count += 1 

xs = [x for x in range(2, 36, 2)]
ys = [] 
for i in bird_data:
    for (j, z) in zip(i, range(2, 72, 2)):
        if j >= 0.50:
            ys.append(z)
            break

square = np.zeros(shape=(32, 32))
for i in range(32):
    for j in range(32):
        square[i][j] = bird_data[i][j]
W = np.tril(square.T) + np.triu(square, 1)
for i in range(32):
    for j in range(32):
        bird_data[i][j] = W[i][j]

In [ ]:
def func(x, a, m):
    return m* x**a
model_bird, _ = curve_fit(func, xs, ys)
print(model_bird)

In [ ]:
styles = ['/home/tqtraaqs1/Experiment/experiment/modules/LLRS/config/TQT-RAAQS.mplstyle']
with plt.style.context(styles):
    fig = plt.figure()
    gs = GridSpec(1,1, figure=fig)
    fig.add_subplot(gs[0, 0])
    ax=fig.axes

    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 'size': 7})
    fontsize = 7
    plt.rc('pdf', fonttype=42) # make the test exportable as text, not an image

    # Calculate figure size in inches and set the size of the figure
    figure_width_cm = 4.3
    figure_height_cm = 4.3
    figure_width_in = figure_width_cm / 2.54
    figure_height_in = figure_height_cm / 2.54
    fig.set_size_inches((figure_width_in, figure_height_in))
    margins = {  #     vvv margin in inches
    "left"   :     0.75 / 4.8,
    "bottom" :     0.5 / 4.8,
    "right"  : 1 - 0.75 / 4.8,
    "top"    : 1 - 0.4   / 4.8 
    }
    fig.subplots_adjust(**margins)
    im = ax[0].imshow(np.rot90(bird_data), extent=[1, 65, 1, 73], cmap='RdBu')
    pixel_size = 0.8 
    fmt.format_plot(ax[0], plot_params={
        'xticks': np.arange(0, 65, 8),  # Centered ticks for x-axis
        'yticks': np.arange(0, 73, 8),
        'xticklabels': np.arange(0,65,8),
        'yticklabels': np.arange(0,73,8),
        'xlim_epsilon': 0.075,
        'ylim_epsilon': 0.075,
        'xlim': (0,64),
        'ylim': (0,72),
        'ytitle': 'Nty',
        'xtitle': 'Ntx'
    })
    #ax[0].plot(X.ravel(), Y.ravel(), c='black', markersize=1)
    

    for k in [512, 1024, 2048]:
        xp = np.linspace(k/72, 64, 65)
        yp = k/xp
        fmt.plot_curve(ax[0], xp, yp, series_params={'linestyle': '-', 'primary': 'lightgrey'})
    xcurve = np.linspace(0, 50, 1000, dtype=int)
    y_sp = model_bird[1]*(xcurve)**model_bird[0]
    fmt.plot_curve(ax[0], xcurve, y_sp, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})
    fmt.plot_curve(ax[0], y_sp, xcurve, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})
 

    divider = fmt.make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cb=fig.colorbar(im, cax=cax, fraction=0.006)
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_label('Mean success probability')

now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
filename = f"rrcv3_mean_success_surface_plot.pdf"
fig.savefig(filename)
plt.show()  

### RedRec V2 Surface Plot

In [ ]:
list_of_bird_files = [f for f in os.listdir(str(project_dir) + "/resources/operational-benchmarking/redrec-v2/")]
list_of_bird_files.sort(key= lambda x: int(x.split("_")[3]))
bird_data = np.zeros(shape=(32, 36))
count = 0
for itr in list_of_bird_files:
    if count == 32:
        break
    with open(str(project_dir) + "/resources/operational-benchmarking/redrec-v2/" + itr, 'rb') as f:
        data = pickle.load(f)
        data.sort(key = lambda x: x[0])
        data = [x for x in data if x[0] < 74]
        offset = 36 - len(data)
        zeros = [((x+1)*2,0) for x in range(offset)]
        data = zeros + data

    bird_data[count] = [x[1] for x in data]
    count += 1 

xs = [x for x in range(2, 28, 2)]
ys = [] 
for i in bird_data:
    for (j, z) in zip(i, range(2, 74, 2)):
        if j >= 0.50:
            ys.append(z)
            break

square = np.zeros(shape=(32, 32))
for i in range(32):
    for j in range(32):
        square[i][j] = bird_data[i][j]
W = np.tril(square.T) + np.triu(square, 1)
for i in range(32):
    for j in range(32):
        bird_data[i][j] = W[i][j]

In [ ]:
def func(x, a, m):
    return m* x**a
model_bird, _ = curve_fit(func, xs, ys) 

In [ ]:
styles = ['/home/tqtraaqs1/Experiment/experiment/modules/LLRS/config/TQT-RAAQS.mplstyle']
with plt.style.context(styles):
    fig = plt.figure()
    gs = GridSpec(1,1, figure=fig)
    fig.add_subplot(gs[0, 0])
    ax=fig.axes

    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 'size': 7})
    fontsize = 7
    plt.rc('pdf', fonttype=42) # make the test exportable as text, not an image


    # Calculate figure size in inches and set the size of the figure
    figure_width_cm = 4.3
    figure_height_cm = 4.3
    figure_width_in = figure_width_cm / 2.54
    figure_height_in = figure_height_cm / 2.54
    fig.set_size_inches((figure_width_in, figure_height_in))
    margins = {  #     vvv margin in inches
    "left"   :     0.75 / 4.8,
    "bottom" :     0.5 / 4.8,
    "right"  : 1 - 0.75 / 4.8,
    "top"    : 1 - 0.4   / 4.8 
    }
    fig.subplots_adjust(**margins)
    im = ax[0].imshow(np.rot90(bird_data), extent=[1, 65, 1, 73], cmap='RdBu')
    pixel_size = 0.8 
    fmt.format_plot(ax[0], plot_params={
        'xticks': np.arange(0, 65, 8),  # Centered ticks for x-axis
        'yticks': np.arange(0, 73, 8),
        'xticklabels': np.arange(0,65,8),
        'yticklabels': np.arange(0,73,8),
        'xlim_epsilon': 0.075,
        'ylim_epsilon': 0.075,
        'xlim': (0,64),
        'ylim': (0,72),
        'ytitle': 'Nty',
        'xtitle': 'Ntx'
    })

    im = ax[0].imshow(np.rot90(bird_data), extent=[1, 65, 1, 73], cmap='RdBu')
    for k in [512, 1024, 2048]:
        xp = np.linspace(k/72, 64, 65)
        yp = k/xp
        fmt.plot_curve(ax[0], xp, yp, series_params={'linestyle': '-', 'primary': 'lightgrey'})
    xcurve = np.linspace(0, 50, 1000, dtype=int)
    
    y_sp = model_bird[1]* xcurve**model_bird[0] 
    fmt.plot_curve(ax[0], xcurve, y_sp, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})
    fmt.plot_curve(ax[0], y_sp, xcurve, series_params={'linestyle': '-', 'primary': 'black', 'linewidth' : 0.7})


    divider = fmt.make_axes_locatable(ax[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cb=fig.colorbar(im, cax=cax, fraction=0.006)
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_ticks(np.linspace(0,1,6))
    cb.set_label('Mean success probability')

now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
filename = f"rrcv2_mean_success_surface_plot.pdf"
fig.savefig(filename)
plt.show()  